In [2]:
import pandas as pd

In [3]:
df = pd.read_json('new_data.jsonl', lines=True)

In [7]:
output = []

### Get Current Code and Prompt

In [4]:
idx = 75

with open('0-prompt.txt', 'w') as writefile:
    writefile.write(df.iloc[idx]['messages'][1]['content'])
    
with open('0-code.txt', 'w') as writefile:
    writefile.write(df.iloc[idx]['messages'][2]['content'])

### Write to Output

In [111]:
prompt = [{"role": "system", "content": "You are the greatest Smart contract Ethereum developer in the world. You have never messed up and always write the best and most complicated yet functioning Solidity contracts."}]

In [112]:
user = """Generate a PerkyPineapples smart contract using Solidity 0.4.17. Fill in all the functions using the function signatures and accomplish the following requirements:
Task Description: Develop a unique ERC721 token smart contract, PerkyPineapples, designed to manage a collection of NFTs with distinct minting phases, including a free minting period. This contract incorporates several adjustable parameters to control the sales process, such as activation status, price per token, maximum tokens per transaction, and limitations on free mints per wallet. It also includes a mechanism to gift tokens and withdraw contract balances, alongside integration with proxy registries for marketplace compatibility.

Contract Specifications:
- Inherits ERC721 and Ownable functionalities for NFT management and ownership control.
- Defines parameters for sale activation, token pricing, minting caps, and free mint allowances.
- Utilizes a counter to track the total supply of tokens.
- Allows for the setting of hidden and base URIs to manage how token metadata is presented before and after reveal.
- Includes proxy registry address integration for marketplace facilitation.

Key Functionalities:
1. Sale Activation: The contract owner can toggle the sale's active status, controlling when minting is allowed.
2. Minting: Users can mint a specified number of tokens, adhering to maximum per transaction and total supply limits, with a distinction between paid and free minting based on the current supply and per-wallet free mint limits.
3. Gifting Tokens: The contract owner can mint tokens directly to any address, bypassing payment and mint limits.
4. Withdrawal: Enables the contract owner to withdraw the balance of ETH collected from token sales.
5. Metadata Management: Offers functions to set hidden and revealed metadata URIs for the tokens.
6. Marketplace Integration: Adjusts approval checks for token operations through a proxy registry, aiding in marketplace listings.

Implementation Details:
- The contract distinguishes between free and paid mints based on the total amount of tokens already minted and individual wallet contributions to the free mint count.
- Incorporates safety checks to prevent sending ETH to the contract unintentionally.
- Utilizes a constructor to gift an initial set of tokens to the contract deployer.
- Employs Counters to safely increment token IDs upon minting.
- Provides public view functions for querying contract states, such as total supply and sale activation status.

Security Considerations:
- Ensures that minting functions check for sale activation and adhere to max supply constraints to prevent over-minting.
- Uses onlyOwner modifiers for functions that modify critical parameters, ensuring that only the contract owner can alter the sale dynamics.
- Prevents direct ETH transfers to the contract, mitigating the risk of unintended contract interactions.
- Safe use of external calls (msg.sender.call) in the withdrawal function, though care must be taken to monitor for reentrancy vulnerabilities.
"""

In [82]:
code = df.iloc[idx]['messages'][2]['content']

In [113]:
code = """
contract PerkyPineapples is ERC721, Ownable {
    bool public saleIsActive = false;

    uint256 public price = 0.025 ether;
	uint public maxPerTx = 10;
	uint public maxFreeMintPerWallet = 20;
	uint public amountAvailableFreeMint = 200;
	uint public maxSupply = 5000;
	address proxyRegistryAddress ; 
	mapping(address => uint) public addressFreeMinted;
    string public _hiddenURI = "ipfs://QmRkNnfQExTwer7vL1PNuRszzshpUWjRb7AerWQt1Ef82Q/hidden.json";
    string public _baseURIextended = "";
	
    using Counters for Counters.Counter;
    Counters.Counter private _tokenSupply;

    constructor() ERC721("PerkyPineapples", "PP") {
        for (uint256 i = 0; i < 10; i++){
            gift(msg.sender);
        }
    }

    function totalSupply() public view returns (uint256 supply) {
        return _tokenSupply.current();
    }

    function _baseURI() internal view virtual override returns (string memory) {
        return _baseURIextended;
    }

    function isApprovedForAll(address owner, address operator) public view override returns (bool) {
        ProxyRegistry proxyRegistry = ProxyRegistry(proxyRegistryAddress);
        if (address(proxyRegistry.proxies(owner)) == operator) {
            return true;
        }

        return super.isApprovedForAll(owner, operator);
    }

    function mint(uint numberOfTokens) external payable {
        require(saleIsActive, "Sale is inactive.");
        require(numberOfTokens <= maxPerTx, "You can only mint 10 at a time.");
        require(_tokenSupply.current() + numberOfTokens <= maxSupply, "Purchase would exceed max supply of tokens");
        
        if (_tokenSupply.current() + numberOfTokens > amountAvailableFreeMint) {
            require((price * numberOfTokens) <= msg.value, "Don't send under (in ETH).");
        } else {
			require(msg.value == 0, "Don't send ether for the free mint.");
			require(addressFreeMinted[msg.sender] < maxFreeMintPerWallet, "You can only adopt 20 free CAU per wallet. Wait for the paid adoption.");
		}
		
		addressFreeMinted[msg.sender] += numberOfTokens;
        for(uint i = 0; i < numberOfTokens; i++) {
		  uint256 _tokenId = _tokenSupply.current() + 1;
            _safeMint(msg.sender, _tokenId);
            _tokenSupply.increment();
        }
    }

    function gift(address _to) public onlyOwner {
        uint256 _tokenId = _tokenSupply.current() + 1;
        _safeMint(_to, _tokenId);
        _tokenSupply.increment();
    }

    function withdraw() public onlyOwner {
        (bool success, ) = msg.sender.call{value: address(this).balance}("");
        require(success, "Address: unable to send value, recipient may have reverted");
    }

    function tokenURI(uint256 tokenId) public view virtual override returns (string memory) {
        require(_exists(tokenId), "ERC721Metadata: URI query for nonexistent token");

        string memory baseURI = _baseURI();
        return bytes(baseURI).length > 0 
            ? string(abi.encodePacked(baseURI, Strings.toString(tokenId), ".json")) 
            : _hiddenURI; // If the base URI is not set, return a hidden URI
    }

    function setSaleIsActive(bool newState) public onlyOwner {
        saleIsActive = newState;
    }

    function setPrice(uint newPrice) public onlyOwner {
        price = newPrice;
    }

    function setMaxPerTx(uint _maxPerTx) external onlyOwner() {
        maxPerTx = _maxPerTx;
    }

    function setMaxFreeMintPerWallet(uint _maxFreeMint) public onlyOwner() {
        maxFreeMintPerWallet = _maxFreeMint;
    }

    function setAmountAvailableFreeMint(uint _freeMint) public onlyOwner() {
        amountAvailableFreeMint = _freeMint;
    }

    function setMaxSupply(uint _maxSupply) public onlyOwner() {
        maxSupply = _maxSupply;
    }

    // rinkeby: 0xf57b2c51ded3a29e6891aba85459d600256cf317
    // mainnet: 0xa5409ec958c83c3f309868babaca7c86dcb077c1
    function setProxyRegistryAddress(address proxyAddress) external onlyOwner {
        proxyRegistryAddress = proxyAddress;
    }

    function setHiddenUri(string memory hiddenURI) public onlyOwner() {
        _hiddenURI = hiddenURI;
    }

    function setBaseURI(string memory baseURI_) external onlyOwner() {
        _baseURIextended = baseURI_;
    }
}
"""

In [114]:
prompt.append({"role":"user","content":"{}".format(user)})
prompt.append({"role": "assistant", "content": "{}".format(code)})

output.append(prompt)

prompt = []
user = ""
code = ""

In [115]:
import jsonlines

with jsonlines.open('edit-prompts2.jsonl', mode='w') as writer:
    writer.write_all(output)